In [ ]:
!pip install fastapi uvicorn datasets scikit-learn joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nv

In [ ]:
!pip install pyngrok

In [ ]:
# Add this code to the end of your script to create a public URL
# Install required package first if needed
# !pip install pyngrok localtunnel

# Method 1: Using pyngrok (requires authentication)
def setup_ngrok():
    try:
        from pyngrok import ngrok
        # You need to set your authtoken
        # Get it from https://dashboard.ngrok.com/get-started/your-authtoken
        # ngrok.set_auth_token("YOUR_AUTH_TOKEN")

        # Start ngrok tunnel (replace 8000 with your port if different)
        public_url = ngrok.connect(8000).public_url
        print(f"🔗 Public API URL: {public_url}")
        return public_url
    except Exception as e:
        print(f"Error setting up ngrok: {e}")
        return None

# Method 2: Using localtunnel (doesn't require authentication)
def setup_localtunnel():
    import os
    import subprocess
    import json

    try:
        # Check if localtunnel is installed
        result = subprocess.run(["npx", "localtunnel", "--version"],
                                capture_output=True, text=True)
        if "not found" in result.stderr:
            print("Installing localtunnel...")
            os.system("npm install -g localtunnel")

        # Start localtunnel (replace 8000 with your port if different)
        print("Starting localtunnel...")
        process = subprocess.Popen(
            ["npx", "localtunnel", "--port", "8000"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True
        )

        # Wait a bit for the tunnel to establish
        import time
        time.sleep(3)

        # Read from stderr to get the URL
        for line in process.stderr:
            if "your url is" in line.lower():
                url = line.split("is:")[1].strip()
                print(f"🔗 Public API URL: {url}")
                return url

        print("Localtunnel started but couldn't capture URL. Check terminal output.")
        return None
    except Exception as e:
        print(f"Error setting up localtunnel: {e}")
        return None

# Choose which tunnel you want to use (or use both)
# public_url = setup_ngrok()  # Uncomment this if you have ngrok auth token
# public_url = setup_localtunnel()  # Use this if you don't have ngrok auth token

In [ ]:
import joblib
import uvicorn
import numpy as np
import nest_asyncio
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from fastapi.middleware.cors import CORSMiddleware
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from pydantic import BaseModel
import re
from typing import Optional
import os
import subprocess
import time
import random
import string
import requests
from threading import Thread

# Apply nest_asyncio to allow running an event loop inside another event loop
nest_asyncio.apply()

# Define Pydantic models for requests
class CodeInput(BaseModel):
    code: str

class CodeOutput(BaseModel):
    prediction: str
    confidence: float
    has_error: bool
    fixed_code: Optional[str] = None
    error_details: Optional[str] = None

# Initialize FastAPI
app = FastAPI()

# Add CORS middleware to allow all origins
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Create templates directory and HTML file for the web interface
if not os.path.exists("templates"):
    os.makedirs("templates")

# Create a simple HTML interface
with open("templates/index.html", "w") as f:
    f.write("""<!DOCTYPE html>
<html>
<head>
    <title>Bug Detection and Correction API</title>
    <style>/* Your existing HTML content */</style>
</head>
<body>
    <h1>Bug Detection and Correction API</h1>
    <p>Enter your code below to check for bugs and get suggestions for fixing it:</p>

    <textarea id="codeInput" placeholder="Paste your code here..."></textarea>
    <button onclick="analyzeCode()">Analyze Code</button>

    <div id="result" class="result hidden">
        <h2>Analysis Result</h2>
        <p id="prediction"></p>
        <p id="confidence"></p>
        <div id="errorDetails" class="error hidden"></div>

        <h3>Fixed Code:</h3>
        <textarea id="fixedCode" readonly></textarea>
        <button onclick="copyToClipboard()">Copy Fixed Code</button>
    </div>

    <script>/* Your existing script */</script>
</body>
</html>""")

# Set up templates
templates = Jinja2Templates(directory="templates")

# Load or train model
def load_or_train_model():
    try:
        model = joblib.load("bug_fix_model.pkl")
        vectorizer = joblib.load("vectorizer.pkl")
        print("Loaded existing model and vectorizer")
    except FileNotFoundError:
        print("Training new model...")
        dataset = load_dataset("google/code_x_glue_cc_clone_detection_big_clone_bench")
        df = dataset["train"].to_pandas()
        sample_size = min(10000, len(df))
        df = df.sample(n=sample_size, random_state=42)
        X = df["func1"] + " " + df["func2"]
        y = df["label"].astype(int)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        vectorizer = CountVectorizer(max_features=10000)
        X_train_vec = vectorizer.fit_transform(X_train)
        model = LinearSVC(dual=False)
        model.fit(X_train_vec, y_train)
        joblib.dump(model, "bug_fix_model.pkl")
        joblib.dump(vectorizer, "vectorizer.pkl")
        X_test_vec = vectorizer.transform(X_test)
        accuracy = model.score(X_test_vec, y_test)
        print(f"Model accuracy: {accuracy:.4f}")
    return model, vectorizer

# Common syntax errors and their fixes
common_errors = [
    (r'(\w+)\s*=\s*(\w+)\s*\+\s*(\d+);', r'\1 = \2 + \3'),
    # Add your regex patterns here
]

def fix_code(code):
    fixed_code = code
    error_details = []
    for pattern, replacement in common_errors:
        matches = re.findall(pattern, fixed_code)
        if matches:
            fixed_code = re.sub(pattern, replacement, fixed_code)
            error_details.append(f"Fixed pattern: {pattern}")
    return fixed_code, error_details

# Setup tunnel using ngrok
def setup_ngrok():
    try:
        from pyngrok import ngrok
        public_url = ngrok.connect(8000).public_url
        print(f"🔗 Public API URL: {public_url}")
        return public_url
    except Exception as e:
        print(f"Error setting up ngrok: {e}")
        return None

# Setup tunnel using localtunnel
def setup_localtunnel():
    try:
        result = subprocess.run(["npx", "localtunnel", "--version"], capture_output=True, text=True)
        if "not found" in result.stderr:
            print("Installing localtunnel...")
            os.system("npm install -g localtunnel")
        print("Starting localtunnel...")
        process = subprocess.Popen(["npx", "localtunnel", "--port", "8000"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
        time.sleep(3)
        for line in process.stderr:
            if "your url is" in line.lower():
                url = line.split("is:")[1].strip()
                print(f"🔗 Public API URL: {url}")
                return url
        return None
    except Exception as e:
        print(f"Error setting up localtunnel: {e}")
        return None

def setup_tunnel():
    try:
        print("Setting up localtunnel...")
        process = subprocess.Popen(
            ["npx", "localtunnel", "--port", "8000"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True
        )
        time.sleep(5)  # give it a moment to start

        # Read stdout lines until URL is found
        while True:
            output = process.stdout.readline()
            if output == '' and process.poll() is not None:
                break
            if "url is:" in output.lower() or "your url is:" in output.lower():
                url = output.strip().split()[-1]
                print(f"\n🔗 ACCESS YOUR API HERE: {url}\n")
                break
    except Exception as e:
        print(f"Error setting up tunnel: {e}")


# FastAPI routes
@app.get("/", response_class=HTMLResponse)
async def get_html(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

@app.post("/predict", response_model=CodeOutput)
async def predict(code_input: CodeInput):
    fixed_code, error_details = fix_code(code_input.code)
    transformed_code = vectorizer.transform([code_input.code])
    prediction = model.predict(transformed_code)[0]
    confidence = abs(model.decision_function(transformed_code)[0])
    result = "Buggy Code" if prediction == 1 else "Fixed Code"
    has_error = prediction == 1
    return CodeOutput(
        prediction=result,
        confidence=float(confidence),
        has_error=has_error,
        fixed_code=fixed_code if has_error or error_details else code_input.code,
        error_details=", ".join(error_details) if error_details else None
    )

# Start the FastAPI server
def start_server():
    setup_tunnel()  # Use localtunnel
    # setup_ngrok()  # Uncomment this if you prefer ngrok
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    print("Starting Bug Detection API on http://localhost:8000")
    print("You can access the web interface v ia the tunnel URL printed above")
    server.run()

if __name__ == "__main__":
    model, vectorizer = load_or_train_model()
    start_server()


Loaded existing model and vectorizer
Setting up localtunnel...

🔗 ACCESS YOUR API HERE: https://real-animals-rhyme.loca.lt



INFO:     Started server process [570]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Starting Bug Detection API on http://localhost:8000
You can access the web interface v ia the tunnel URL printed above
